# **<center> 💻 ANALIZADOR GRAMATICO (COMPILADORES) 💻 </center>**
------
### Datos Academicos 📖
- **Institucion:** Universidad Nacional de San Antonio Abad del Cusco
- **Facultad:** Facultad de ingenieria electrica, electronica, informatica y mecanica
- **Escuela Prof:** Ingenieria Informatica y de Sistemas

#### Docente: 
- **Vanesa Lavilla Alvarez** - _Docente_ - [Concytec](http://directorio.concytec.gob.pe/appDirectorioCTI/VerDatosInvestigador.do;jsessionid=c5a14b2abf78a327623f1a7ccd0f?id_investigador=114602).

#### Trabajo:
- Realizar un analizador gramatico que dado un conjunto de reglas de entrada pueda analizar las reglas, en busca de problemas de recursion o ambiguedad, para luego poder corregirlas y mostrar el resultado.

#### Autores:✒️

- **Widmar Raul Quispe Leon** - _GitHub Account_ - [WidmarO](https://github.com/WidmarO)
- **Melanie Indira Sullca Peralta** - _GitHub Account_ - [Melanie279](https://github.com/Melanie279)
- **Nadiabeth Diana Mallqui Apaza** - _GitHub Account_ - [Nadiabeth15](https://github.com/Nadiabeth15)
------
NOTA: Diculpe las faltas ortograficas tales como tildes, dieresis y demas faltas encontradas a lo largo del cuaderno, esto se debe a que se uso unicamente una distribucion de teclado en ingles para escribir el presente el cual no tiene acceso a dichos caracteres.

## Empezamos... 🚀
Para un mayor entendimiento del programa realizamos un diagrama de flujo, el cual muestra una secuencia de pasos que componen el proceso del programa.

### Ejemplos...

Dado el siguiente conjunto de reglas determinar si tienen problemas de ambiguedad o recursividad, si es el caso corregirlos.

```py
# Conjunto de reglas
S -> Var := E
E -> E + E | E + (E) | (E)
E -> E - T
E -> Var | Num
Var -> a | b | c | d | e
Num -> 0 | 1 | 2 | 3 | 9
# Tiene Recursividad y Ambiguedad
# Corregimos Recursividad
S -> Var := E
E -> (E) E' | Var E' | Num E'
E' -> + E E' | + (E) E' | - T E' | ε
Var -> a | b | c | d | e
Num -> 0 | 1 | 2 | 3 | 9
# Aun tiene problemas de Ambiguedad
# corregimos ambiguedad
S -> Var := E
E -> (E) E' | Var E' | Num E'
E' -> + E'' | - T E' | ε
E'' -> E E' | (E) E'
Var -> a | b | c | d | e
Num -> 0 | 1 | 2 | 3 | 9
# Ahora nuestras reglas no tienen problemas de ambiguedad ni recursividad :)
```


## Diagrama de flujo

![diagrama.png](https://raw.githubusercontent.com/WidmarO/Analizador-compiladores/master/img/diagrama.png)

### Construido con... 🛠️
* Lenguaje: [Python 3.6.9](https://www.python.org/)
* Servicio Cloud: [Google Colaboratory](https://colab.research.google.com/notebooks/intro.ipynb)
* Servicio Local: [Jupyter-Notebook](https://jupyter.org/)
* Diagrama de flujo: [Visual Paradigm](https://www.visual-paradigm.com/)
* Interfaz Grafica: [tkinter](https://docs.python.org/2/library/tkinter.html)
* Editor: [Visual Studio Code](https://code.visualstudio.com/)

## Codificacion del programa 📄

A continuacion mostramos los modulos mas importantes del programa...

### Modulo que recibe los datos 
Ingresadas las reglas el modulo LeerDatos recibe como parametro las reglas en un string separadas las lineas por "\n" y devuelve un arreglo con las reglas separadas por lineas.


In [1]:
# E -> E + E | E + (E) | (E)
# E -> E - T
# E -> Var | Num
# Var -> a | b | c
# Num -> 0 | 1 | 2

Datos_de_entrada = "E -> E + E | E + (E) | (E)\nE -> E - T\nE -> Var | Num\nVar -> a | b | c\nNum -> 0 | 1 | 2"

def LeerDatos(entrada):
  reglas = entrada.split("\n")
  reglas = [item for item in reglas if item]
  return reglas

print(LeerDatos(Datos_de_entrada),"\n")
reglas = LeerDatos(Datos_de_entrada)
for i in reglas:
    print(i)

['E -> E + E | E + (E) | (E)', 'E -> E - T', 'E -> Var | Num', 'Var -> a | b | c', 'Num -> 0 | 1 | 2'] 

E -> E + E | E + (E) | (E)
E -> E - T
E -> Var | Num
Var -> a | b | c
Num -> 0 | 1 | 2


### Modulo que estructura la informacion
Ya recuperados los datos en un arreglo, debemos organizar la informacion para poder tratar los datos de mejor manera; este modulo recibe como parametro el arreglo con las reglas obtenidas y retorna un diccionario donde las llaves son los estados no terminales y los valores son arreglos con subarreglos con las reglas separando el primer estado de los demas.

In [2]:
def OrganizarToAnalisis(reglas):
  dic = {}
  for row in reglas:
    aux = row.split(" -> ",1)
    premisa = aux[0]
    dic[premisa] = []

  for row in reglas:
    aux = row.split(" -> ",1) 
    premisa = aux[0]
    derecha = aux[1]
    arreglo = derecha.split(" | ")
    for token in arreglo:      
      aux = token.split(' ',1)
      dic[premisa].append(aux)
  return dic
#
diccionario1 = OrganizarToAnalisis(reglas)
for i in diccionario1:
    print(i,":",diccionario1[i])

E : [['E', '+ E'], ['E', '+ (E)'], ['(E)'], ['E', '- T'], ['Var'], ['Num']]
Var : [['a'], ['b'], ['c']]
Num : [['0'], ['1'], ['2']]


### Modulo que Analiza si existe problemas de recursion
Este modulo recibe como parametro un diccionario obtenido del modulo anterior, analiza segun esa estructura y retorna una lista con los estados en los que se ha encontrado algun error de recursion, si la lista esta vacia es por que no existe errores de recursion.

In [3]:
def AnalizarForRec(dic):
  errores = []
  for e in dic:
    for token in dic[e]:
      if (token[0] == e):
        # print("Existe Recursividad")
        errores.append(e)
  errores = list(set(errores))

  if (len(errores) > 0):
    return errores
  else:
    # print("No hay problemas de Recursividad")
    return errores
#
err_rec = AnalizarForRec(diccionario1)
if(len(err_rec) > 0):
    print("Existe errores de recursion en: ", err_rec)
else:
    print("No existe errores de recursion: ",err_rec)

Existe errores de recursion en:  ['E']


### Modulo que Analiza si existe problemas de Ambiguedad
Este modulo recibe como parametro un diccionario estructurado como el que devuelve el modulo "OrganizarToAnalisis", analiza segun esa estructura y retorna un diccionario con los estados en los que se ha encontrado algun error de ambiguedad como valores y como llaves el estado que ocaciona esa ambiguedad con mayor repeticiones, si el diccionario esta vacio es por que no existe errores de recursion.

In [4]:
def AnalizarForAmb(dic):
  errores = {}
  aux = {}
  for e in dic:
    aux[e] = {}
    for token in dic[e]:
      aux[e][token[0]] = 0

  for e in dic:
    mayor = 0
    letra = ""
    for token in dic[e]:
      aux[e][token[0]] += 1
      if (aux[e][token[0]] > mayor and aux[e][token[0]] > 1):
        mayor = aux[e][token[0]]
        letra = token[0]

    if (letra != ""):
      errores[e] = letra

  if (len(errores) > 0):
    # print("Existe Ambiguedad en " + e)
    return errores
  else:
    # print("No hay problemas de Ambiguedad")
    return errores
#
err_amb = AnalizarForAmb(diccionario1)
if (len(err_amb) > 0):
    print("Existe errores de Ambiguedad:",err_amb)
else:
    print("No existen errores de Ambiguedad:",err_amb)

Existe errores de Ambiguedad: {'E': 'E'}


### Modulo que corrige los problemas de recursion
Este modulo corrige los problemas de recursion encontrados al analizar las reglas con el modulo "AnalizarForRec", recibe como parametro un diccionario estructurado para la correcion y retorna un diccionario corregido.

In [5]:
def CorregirRec(dic):
  err = AnalizarForRec(dic) #devuelve una lista
  corregido = {}
  for e in dic:
    if e in err:
      corregido[e] = []
      corregido[e + "'"] = []
      for token in dic[e]:
        if (len(token) == 1):
          corregido[e].append([token[0] + " " + e + "'"])
        else:
          corregido[e + "'"].append([(token[1]) + " " + e + "'"])
      corregido[e + "'"].append(["\u03B5"])
    else:
      corregido[e] = dic[e]  
  return corregido

diccionario2 = CorregirRec(diccionario1)
for i in diccionario2:
    print(i,":",diccionario2[i])

E : [["(E) E'"], ["Var E'"], ["Num E'"]]
E' : [["+ E E'"], ["+ (E) E'"], ["- T E'"], ['ε']]
Var : [['a'], ['b'], ['c']]
Num : [['0'], ['1'], ['2']]


### Modulo que corrige los problemas de ambiguedad
El modulo "CorregirAmb" hace uso de otros dos modulos que juntos solucionan los problemas de ambiguedad encontrados en el analisis de las reglas con el modulo "AnalizarForAmb", como parametro de entrada recibe un diccionario estructurado y retorna un diccionario corregido.

In [6]:
def CorregirUnaAmb(dic):
  err = AnalizarForAmb(dic)
  corregido = {}
  for e in dic:
    # print("esete es el e: ",e)
    if e in err:
      # print("entro en e.values", e)
      corregido[e] = []
      corregido[e + "'"] = []
      corregido[e].append([err[e] + " " + e + "'"])
      for token in dic[e]:
        if (token[0] != err[e]):
          corregido[e].append(token)
        else:
          corregido[e + "'"].append([token[1]])      
    else:
      corregido[e] = dic[e]  
  return corregido

def Reorganizar(dic):
  for e in dic:
    for i in range(len(dic[e])):
      if (len(dic[e][i]) == 1):
        dic[e][i] = dic[e][i][0].split(' ',1)
  return dic

def CorregirAmb(dic):
  dic3 = Reorganizar(dic)
  err = AnalizarForAmb(dic)
  dic3 = dic
  while(len(err) != 0):
    err = AnalizarForAmb(dic3)
    dic3 = CorregirUnaAmb(dic3)
    dic3 = Reorganizar(dic3)
  return dic3

diccionario3 = CorregirAmb(diccionario2)
for i in diccionario3:
    print(i,diccionario3[i])

E [['(E)', "E'"], ['Var', "E'"], ['Num', "E'"]]
E' [['+', "E''"], ['-', "T E'"], ['ε']]
E'' [['E', "E'"], ['(E)', "E'"]]
Var [['a'], ['b'], ['c']]
Num [['0'], ['1'], ['2']]


### Modulo que Muestra los valores de los diccionarios
Este modulo toma como parametro de entrada un diccionario estructurado y lo convierte en un string separando las lineas por "\n".

In [7]:
def MostrarCorregido(dic):
  rpta = ""
  for premisa in dic:
    chrs = premisa + " -> "
    for regla in dic[premisa]:
      aux = " ".join(regla)
      chrs += aux + ' | '
    chrs = chrs[:-2] + "\n"
    rpta += chrs
  return rpta

to_show = MostrarCorregido(diccionario3)
auxiliar = to_show.split("\n")
for _ in auxiliar:
    print(_)


E -> (E) E' | Var E' | Num E' 
E' -> + E'' | - T E' | ε 
E'' -> E E' | (E) E' 
Var -> a | b | c 
Num -> 0 | 1 | 2 



### Una captura de la interfaz (Analizador_v1.2 )

- Cada boton toma los datos del display1 (Entrada reglas) asi que el boton <<< actualiza el resultado del display2 en el display1.

![diagrama.png](https://raw.githubusercontent.com/WidmarO/Analizador-compiladores/master/img/interfaz.png)

### Modulo que obtiene los valores de estados No terminales
Este modulo toma como parametro de entrada un diccionario estructurado y lo retorna una lista con los valores de los estados no terminales como elementos.

In [8]:

def NoTerminales(reglas):
  nonTerminals = set()
  for i in reglas:
    aux = i.split(" -> ")
    nonTerminals.add(aux[0])
  nonTerminals = list(nonTerminals)
  return nonTerminals

no_terminales = NoTerminales(reglas)
print(no_terminales)

['Num', 'Var', 'E']


### Modulo que obtiene los valores de estados Terminales
Este modulo toma como parametro de entrada un diccionario estructurado y lo retorna una lista con los valores de los estados terminales como elementos.

In [14]:
def Terminales(dic):
  no_terminales = NoTerminales(dic)
  terminales = []
  for e in dic:
    for token in dic[e]:
      for est in token:
        for x in est.split(' '):          
          terminales.append(x)
  
  for e in terminales:
    if(e[0] == '(' or e[0] == '['):
      terminales.remove(e)
      for i in e:
        terminales.append(i)

  terminales = list(set(terminales))
  for i in no_terminales:
    if (i in terminales):
      terminales.remove(i)
  
  if ('ε' in terminales):
    terminales.remove('ε')

  return terminales

terminales = Terminales(diccionario3)
print(terminales)

['-', 'a', '(', '2', 'c', '1', '+', 'b', ')', 'T', '0']


### Modulo que el conjunto primeros en general.
Este modulo toma como parametro de entrada un diccionario estructurado y retorna un diccionario que tiene como llaves los estados no terminales y como valor para cada uno tiene el conjuntos de primeros de dicho estado.

In [15]:
def PrimerosSimples(dic):
  terminales = Terminales(dic)
  no_terminales = NoTerminales(dic)
  todos = set(terminales + no_terminales)
  conj_primeros = {}

  for e in todos:
    conj_primeros[e] = set()

  for i in terminales:
    if(i == "ε"):
      continue
    else:
      conj_primeros[i].add(i)

  for e in dic:
    for token in dic[e]:
      if (len(token[0]) > 1 and (token[0][0] == "(" or token[0][0] == "[")):
        conj_primeros[e].add(token[0][0])
      else:
        conj_primeros[e].add(token[0])      

  for e in conj_primeros:
    conj_primeros[e] = list(conj_primeros[e])

  return conj_primeros

primeros_simples = PrimerosSimples(diccionario3)
for i in primeros_simples:
    print(i,primeros_simples[i])

- ['-']
( ['(']
a ['a']
2 ['2']
c ['c']
1 ['1']
+ ['+']
b ['b']
) [')']
T ['T']
0 ['0']
Num ['2', '0', '1']
E ['Num', 'Var', '(']
E'' ['(', 'E']
E' ['-', 'ε', '+']
Var ['b', 'c', 'a']


### Modulo que obtiene el conjunto primeros solo de los estados terminales
Este modulo toma como parametro de entrada un diccionario estructurado y lo retorna una lista con los valores de los estados terminales como elementos.

In [16]:
def PrimerosTerminales(dic):
  terminales = Terminales(dic)
  primeros = PrimerosSimples(dic)
  aux = []
  for e in primeros:
    if (e not in terminales):
      aux.append(e)

  for x in aux:
    del primeros[x]

  if ('ε' in primeros):
    del primeros['ε']
  
  return primeros

primeros_terminales = PrimerosTerminales(diccionario3)
for i in primeros_terminales:
    print(i,primeros_terminales[i])

- ['-']
( ['(']
a ['a']
2 ['2']
c ['c']
1 ['1']
+ ['+']
b ['b']
) [')']
T ['T']
0 ['0']


### Modulo que obtiene el conjunto primeros solo para los estados no terminales
Este modulo toma como parametro de entrada un diccionario estructurado y lo retorna una lista con los valores de los estados terminales como elementos

In [17]:
def PrimerosNoTerminales(dic):
  no_terminales = NoTerminales(dic)
  terminales = Terminales(dic)
  primeros = PrimerosSimples(dic)
  ans = {}

  aux = []
  for e in terminales:
    # if (e not in no_terminales):
    aux.append(e)
  
  for x in aux:
    # del ans[x]
    del primeros[x]
  
  for e in primeros:
    ans[e] = []
    for est in primeros[e]:
      if(est in primeros):
        # primeros[e].remove(est)
        for i in primeros[est]:
          ans[e].append(i)
      else:
        ans[e].append(est)
  return ans

primeros_no_terminales = PrimerosNoTerminales(diccionario3)
for i in primeros_no_terminales:
    print(i,primeros_no_terminales[i])

Num ['2', '0', '1']
E ['2', '0', '1', 'b', 'c', 'a', '(']
E'' ['(', 'Num', 'Var', '(']
E' ['-', 'ε', '+']
Var ['b', 'c', 'a']
